### Инициализация

In [24]:
#что бы подключить гугл диск к проекту
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
%cd drive/My\ Drive

[Errno 2] No such file or directory: 'drive/My Drive'
/content/drive/My Drive


In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# класс зданиe

In [27]:
class Building():
  def __init__(self, coordinates, distance, azimuth, population, amenity = None):
        self.coordinates = coordinates
        self.distance = distance
        self.azimuth = azimuth
        self.population = population
        self.amenity = amenity



# Поиск средней координаты


In [28]:
def getMediumCoorinates(input: []):
  firstC = 0
  secondC = 0
  for coord in input:
    firstC += coord[0]
    secondC += coord[1]
  return (firstC/len(input), secondC/len(input))


# **Поиск географического центра владивостока:**


In [30]:
vladivostok = open('/content/drive/My Drive/data/Vladivostok_buildings_POI_1/Vladivostok_adm.geojson', 'r')
vlad = json.loads(vladivostok.read())
centreOfVlad = getMediumCoorinates(vlad["features"][0]["geometry"]["coordinates"][0])
print(centreOfVlad)

(131.9971945870371, 43.15664895185186)


In [31]:
!pip install haversine
from haversine import haversine

In [32]:
import math
def azimuth(x1,y1,x2,y2):
  a = 90 - math.atan2(y2-y1, x2-x1)*180/math.pi
  if (a<0): 
    a += 360
  return a


# Перевод всех зданий в полярную СК


In [33]:
buildings= open('/content/drive/My Drive/data/Vladivostok_buildings_POI_1/buildings.geojson', 'r')
building = json.loads(buildings.read())
houses = []
allDistances = []

for i in building["features"]:
  if i["properties"]["population"] != None:
    allDistances.append( haversine(centreOfVlad, i["geometry"]["coordinates"][0][0][0]))
    houses.append(Building(i["geometry"]["coordinates"][0][0][0] ,
                           haversine(centreOfVlad, i["geometry"]["coordinates"][0][0][0]), 
                           azimuth(centreOfVlad[0], centreOfVlad[1], i["geometry"]["coordinates"][0][0][0][0], i["geometry"]["coordinates"][0][0][0][1]), 
                           i["properties"]["population"],
                           str(i["properties"]["amenity_place"] )))

POIs = open('/content/drive/My Drive/data/Vladivostok_buildings_POI_1/POI.geojson', 'r')
POI = json.loads(POIs.read())
for i in POI["features"]:
  houses.append(Building(i["geometry"]["coordinates"] ,
                           haversine(centreOfVlad, i["geometry"]["coordinates"]), 
                           azimuth(centreOfVlad[0], centreOfVlad[1], i["geometry"]["coordinates"][0], i["geometry"]["coordinates"][1]), 
                           100,
                           str(i["properties"]["amenity"] )))


AllSortedDistances = sorted(allDistances)
buildingsByDistance = sorted(houses, key=lambda house: house.distance)



# Поиск точек внутри круга:

In [34]:
import numpy as np

def find_angle(distance,r=0.5):
    angle = math.asin(r/distance)
    return angle * 180/math.pi

def IndexesInRadius(buildingParameter, sortedDistances: [], radius = 0.5): 
  maxIndex = np.argmin(np.abs(np.array(sortedDistances)- buildingParameter - radius))
  minIndex = np.argmin(np.abs(np.array(sortedDistances)- buildingParameter + radius))
  return (minIndex, maxIndex+1)

def GetIndexesInKm(building: Building):
  sortedByAzimuth = []
  allInRing = IndexesInRadius(building.distance, AllSortedDistances)
  sortedInRing = sorted(buildingsByDistance[allInRing[0]:allInRing[1]] , key=lambda house: house.azimuth)
  for i in sortedInRing:
    sortedByAzimuth.append(i.azimuth)

  borderIndexes = IndexesInRadius(building.azimuth, sortedByAzimuth, find_angle(building.distance))
  return borderIndexes , sortedInRing

def GetBuildingsAroundPoint(building: Building):
  indexes = GetIndexesInKm(building)
  borders = indexes[0]
  array = indexes[1]
  return array[borders[0]:borders[1]]


# for i in GetBuildingsAroundPoint(buildingsByDistance[1111]):   #TEST
#   print(i.coordinates, i.distance, i.azimuth)

# Подсчет весов для каждой точки


In [47]:
list_of_places_with_weight={"None":0.7, "bus_station":0, "customs":0, "atm":0.9, "cinema":0.1, "ferry_terminal":0,
                            "nightclub":0, "library":0, "parking":0, "courthouse":0, "fire_station":0, "cafe":0.25,
                            "bank":0.75, "dentist":0, "driving_school":0, "pharmacy":0.1, "post_office":-100, "fuel":0, 
                            "doctors":0, "restaurant":0.2, "veterinary":0, "car_wash":0, "hospital":0.4, "school":0, 
                            "place_of_worship":0.1, "fast_food":0.5, "clinic":0, "Issuing orders":0, "pub":0, 
                            "waste_disposal":0, "toilets":0.2, "bar":0, "police":0, "telephone":0, "waste_basket":0, 
                            "beauty":0, "ski_rental":0, "equipment_rental":0, "theatre":0, "kindergarten":0, 
                            "arts_centre":0.15, "community_centre":0.6, "vending_machine":0.7, "social_facility":0,
                            "bench":0, "shelter":0, "public_building":0, "bbq":0, "post_box":0, "drinking_water":0,
                            "grit_bin":0, "vehicle_ramp":0, "university":0.55, "food_court":0.58,
                            "payment_terminal":0.95, "studio":0, "marketplace":0.75, "fountain":0, "recycling":0, 
                            "public_bookcase":0, "townhall":0, "car_rental":0, "parking_entrance":0, 
                            "vehicle_inspection":0, "childcare":0, "boat_rental":0, "clock":0, "shower":0,
                            "embassy":0, "college":0.45, "charging_station":0, "hunting_stand":0, 'rescue_station': 0, 
                            "bureau_de_change":0, "boat_storage":0, "yacht_club":0, "music_school":0, "stripclub":0, "prison": 0}

buildingWithWeight= {}

for mainBuilding in buildingsByDistance:
  sum = 0
  for buildingAround in GetBuildingsAroundPoint(mainBuilding):
    if (mainBuilding in buildingWithWeight): 
      buildingWithWeight[mainBuilding] += (buildingAround.population % 50)*list_of_places_with_weight[mainBuilding.amenity]
    else:
      buildingWithWeight[mainBuilding] = buildingAround.population*list_of_places_with_weight[mainBuilding.amenity]
      


# Получение искомых координат


In [46]:
def GetNeededPosts():
  sortedBestPoints = {k: v for k, v in sorted(buildingWithWeight.items(), key=lambda item: item[1] , reverse=True)}
  actualPosts = [] 
  for potential in sortedBestPoints:
    flag = False
    for actual in actualPosts:
      if(math.sqrt( (actual[0] - potential.coordinates[0])**2 + (actual[1] - potential.coordinates[1])**2 ) < 0.025 ):
        flag = True
        break

    if flag: continue

    actualPosts.append(potential.coordinates)
    if  sortedBestPoints[potential] < 300: break
  return actualPosts
  
for i in GetNeededPosts():
  print( i, end = ', '  )

[131.9619205, 43.0866066], [131.9099272, 43.1706004], [131.9152868, 43.1293266], [131.9270589, 43.0904242], [131.9448458, 43.1168468], [131.8821826, 43.11465], [131.95233, 43.162462], [131.970601, 43.1364875], [131.9254203, 43.1930143], [131.8615743, 43.0991291], [131.9038145, 43.100341], [131.9979515, 43.2308594], [132.0263614, 43.2443715], [132.0389689, 43.2722172], 